<a href="https://colab.research.google.com/github/ragalaa/Machine-Learning-Basics/blob/master/PYtorchTransferlearning_Imagerecogniszition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch ## for pytorch
import torchvision ## for transfer learnhing models and many other vision related classes
from torch import nn ## Core Neural Network Model classes in Pytorch
from torch import optim ## Contains several Pytorch optimizer classes
import torch.nn.functional as F ## Contains several utilily functions provided by Pytorch

from torchvision import datasets, transforms, models ## Many Computer Vision related classes
                                                     ## for datasets and transformations etc.
from torch.utils.data import * ## Contains several utilily functions for dataset manipulation
from PIL import Image
import numpy as np

In [0]:
#### https://blogs.oracle.com/datascience/transfer-learning-in-pytorch%2c-part-1%3a-how-to-use-dataloaders-and-build-a-fully-connected-class

In [3]:
train_dataset = datasets.CIFAR10('Cifar10', train=True,
                              download=True)

test_dataset = datasets.CIFAR10('Cifar10', train=False,
                             download=True)

0it [00:00, ?it/s]

170500096it [00:02, 69985332.61it/s]                               


Extracting Cifar10/cifar-10-python.tar.gz to Cifar10
Files already downloaded and verified


This gives us two dataset objects that are of torchvision.datasets.cifar.CIFAR10 type. This is a subclass of the PyTorch dataset class, which is the main class to generically represent any dataset. This particular class represents the CIFAR-10 data stored in its internal data structure. Later, these objects shall be passed to a PyTorch Dataloader objects (explained later) for processing the images.

We can verify the lengths (number of images) of both datasets

In [4]:
len(train_dataset),len(test_dataset)

(50000, 10000)

Preprocess the Dataset and Prepare It for Training
Understand the concept of DataLoader and the PyTorch DataLoader API. Split the images into train, validation, and test sets. Create PyTorch DataLoaders to feed images while training, validation, and prediction. Use PyTorch API to define transforms for preprocessing the dataset for more effective training. Use PyTorch API to convert all images to PyTorch tensors. Normalize the dataset using mean and standard deviation of images.

DataLoaders
PyTorch DataLoaders are objects that act as Python generators. They supply data in chunks or batches while training and validation. We can instantiate DataLoader objects and pass our datasets to them. DataLoaders store the dataset objects internally.

When the application asks for the next batch of data, a DataLoader uses its stored dataset as a Python iterator to get the next element (row or image in our case) of data. Then it aggregates a batch worth of data and returns it to the application.

The following is an example of calling the DataLoader constructor:

In [0]:
num_train = len(train_dataset)
indices = list(range(num_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50,sampler=SubsetRandomSampler(indices),
                                           num_workers=0)

In [9]:
len(train_loader)

1000

Here we are creating a DataLoader object for our training dataset with a batch size of 50. The sampler parameter specifies the strategy with which we want to sample data while constructing batches.

We have different samplers available in torch.utils.data.sampler. The explanation is straightforward. You can read about them in the Pytorch Documentation here.

The num_workers argument specifies how many processes (or cores) we want to use while loading our data. This provides parallelism while loading large datasets. Default is 0 which means load all data in main process.

DataLoader reports its length in number of batches. Since we created this DataLoader with a batch size of 50 and we had 50,000 images in our train dataset, we have the length of dataloader = 1000 batches.

Splitting Data
Now let's write a function to split our datasets into train, validation, and test sets, and create their corresponding DataLoaders. 

In [0]:
def split_image_data(train_data,
                     test_data=None,
                     batch_size=20,
                     num_workers=0,
                     valid_size=0.2,
                     sampler=SubsetRandomSampler):
    
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = sampler(train_idx)
    valid_sampler = sampler(valid_idx)
    if test_data is not None:
        test_loader = DataLoader(test_data, batch_size=batch_size,
        num_workers=num_workers)
    else:
        train_idx, test_idx = train_idx[split:],train_idx[:split]
        train_sampler = sampler(train_idx)
        test_sampler = sampler(test_idx)
        
        test_loader = torch.utils.data.DataLoader(train_data, 
                                                  batch_size=batch_size,
                                                  sampler=test_sampler,
                                                  num_workers=num_workers)

    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=num_workers)
    
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size, 
                                               sampler=valid_sampler,
                                               num_workers=num_workers)
    
    return train_loader,valid_loader,test_loader

n the above function, test_data can be None in which case it splits train data into train, test, and validation. If test_data is given, it just splits the train set further into train and validation and creates a separate DataLoader from the test set. The function also uses RandomSubsetSampler to shuffle the train and validation set indices. Let's call this function to obtain our DataLoaders.

In [11]:
trainloader,validloader,testloader = split_image_data(train_dataset,test_dataset,batch_size=50)

len(trainloader),len(testloader),len(validloader)

(800, 200, 200)

And we have a nice split with 800 batches in our train set and 200 each in our validation and test sets respectively. 

Preprocessing and Transforming the Dataset
Before we move on to defining our network and start training, we need to preprocess our datasets. Specifically, we need to perform the following steps:

Resize the images to an appropriate size for our models

Perform some basic and most common data augmentation

Convert the image data to PyTorch Tensors

Normalize the image data

Why Do We Want to Resize Images?
Most of our transfer learning models require data to be of at least 224x224 size. The reason for this limitation is that these models are designed with a large number of convolution and pooling layers, finally followed by a fully connected (linear) layer at the end to generate the classification output. By the time the input image reaches the final layer, it has been reduced drastically in size due to the way convolutions and pooling are defined. If the input image was already too small (like 32x32 CIFAR-10 images in our case), it would be too small for the network to produce any significant output. Therefore, these models sort of restrict us to input an image >=224x224.

Please note that we wouldn't have needed resizing if our images were already > 224x224, such as in the case of ImageNet, or if we were to use our own CNN architecture that did not reduce the image size too much while passing it through layers. Resizing smaller images to larger ones (as in our case) creates artifacts that we don't (ideally) want our model to learn. Since our CIFAR-10 images are really small and the transfer learning models we are using have this requirement, we are obliged to resize.

For datasets with larger images, our GPU or CPU memory constraints may become a factor. Therefore, we combine downsizing with increased batch sizes (until we hit the batch size limit) to optimize the model performance and balance the effects of downsizing.

Data Augmentation 
Data augmentation is a common deep learning technique where we modify images on the fly while training the neural network to see additional images flipped or rotated at different axes and angles. This usually results in better training performance since the network sees multiple views of the same image and has a better chance of identifying its class when minimizing the loss function.

Note that the augmented images are not added to the dataset. Rather, they are created during batch generation, so the actual images seen during training will increase even though you don't see the number of images in the datasets increasing. The length and other functions that count the number of images will still give the same answer. We use the two common augmentations below:

RandomHorizontalFlip that flips some of the images around the vertical axis with a probability p that defaults to 0.5 meaning that 50% of the images shall be flipped.

RandomRotation at a specific degree (10 in our case below) that rotates some of them randomly at an angle of 10 degree again with a probability of p which defaults to 0.5.

In [0]:
from torchvision import transforms
train_transform = transforms.Compose([transforms.Resize(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                     ])

In [0]:
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor()])

dataset = datasets.CIFAR10('Cifar10',download=False,transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=50,num_workers=0)

We first create a dataset from full data and then a DataLoader to feed the data in batches of size 50 to our loop. Note that for DataLoader to work, the images have to be converted to a tensor, so that is the only transform we are using.

The function below is a straightforward implementation that calculates the mean and std of each batch and adds them to their cumulative sums, dividing in the end by the total number of batches to get the averages.

In [0]:
def calculate_img_stats_avg(loader):
    mean = 0.
    std = 0.
    nb_samples = 0.
    for imgs,_ in loader:
        batch_samples = imgs.size(0)
        imgs = imgs.view(batch_samples, imgs.size(1), -1)
        mean += imgs.mean(2).sum(0)
        std += imgs.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    return mean,std

Data Normalization
In data normalization, we statistically normalize the pixel values in our images. This mostly results in better training performance and faster convergence. A common way to perform normalization is to subtract the mean of pixel values of the whole dataset from each pixel, and then divide by the standard deviation of the pixels of the whole dataset.

The most common way in transfer learning is to use the mean and std values of the dataset that the original transfer learning model was trained on. However, it may be a good strategy for cases where we don't want to retrain any part of the original model.

If our dataset is large and we want to retrain whole or part of the original model, then we would be better off normalizing with the mean and standard deviation of the dataset in question (CIFAR-10, in our case). However, in most transfer learning tutorials, you'll find that the mean and std values for ImageNet are used.

Below, I give you two functions to calculate the mean and std of a dataset:

1. "calculate_img_stats_avg" is based on DataLoader and calculates means and stds of each batch of data as it is retrieved from the dataset object, and finally takes the average of the accumulated means and std values. Although this gives us an approximation of the actual values, it is reasonable to use for large datasets that won't fit into memory at the same time. This code has been adapted from the PyTorch forum.

2. "calculate_img_stats_full" calculates the actual mean and std of the whole dataset by working on it at once. This gives more accurate values, but will most likely run out of memory for large datasets. For CIFAR-10, this function requires 28GB of RAM. My machine has 32GB but it falls short and I am unable to run this function. This code has been adapted from the book "Deep Learning with PyTorch" by Eli Stevens and Luca Antiga, Manning Publications.

You can try to run the second function on your specific dataset and if you run into memory issues, then revert to the first one for a good approximation. In case of CIFAR-10, however, many people have calculated the mean and std of the dataset and the values are well known, like ImageNet. We are using those values in the code that follows. I did not try with the approximate values given by the first function but you are welcome to do so .

In [18]:
calculate_img_stats_avg(loader)

(tensor([0.4914, 0.4822, 0.4465]), tensor([0.2023, 0.1994, 0.2010]))

In [0]:
def calculate_img_stats_full(dataset):
    imgs_ = torch.stack([img for img,_ in dataset],dim=3)
    imgs_ = imgs_.view(3,-1)
    imgs_mean = imgs_.mean(dim=1)
    imgs_std = imgs_.std(dim=1)
    return imgs_mean,imgs_std

In [20]:
calculate_img_stats_full(dataset)

(tensor([0.4915, 0.4823, 0.4468]), tensor([0.2470, 0.2435, 0.2616]))

The torch.stack function above stacks the data along the given dimension (3, in our case). The view operation views the tensor as a 3 x (product of all other dimensions) which basically flattens while keeping the first dimension as 3. The best way to visualize what is going on in an obscure function like this is to copy isolate the statements and feed them some dummy tensors to see what's going on. I leave it for you as an exercise. Values below have been taken from the same book (referred above from which the code has been taken):

In [0]:
cifar10_mean = [0.4915, 0.4823, 0.4468]
cifar10_std  = [0.2470, 0.2435, 0.2616]m

Now we can create our datasets again from scratch with all the transformations, augmentations, and normalization applied—splitting them into train and test, and obtaining the final DataLoaders. Note that we also define our batch size = 50.

In [22]:
batch_size = 50
train_transform = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(cifar10_mean, cifar10_std)
                                     ])

test_transform = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(cifar10_mean, cifar10_std)
                                    ])

train_data = datasets.CIFAR10('Cifar10', train=True,
                              download=False, transform=train_transform)
test_data = datasets.CIFAR10('Cifar10', train=False,
                             download=False, transform=test_transform)

trainloader,validloader,testloader = split_image_data(train_data,test_data,batch_size=batch_size)

len(trainloader),len(testloader),len(validloader)

(800, 200, 200)

toTensor() converts a numpy array to a PyTorch Tensor (all our images are constructed as numpy arrays by the dataset class when read from disk). normalize() is a transform that normalizes according to the passed values of means and STD of each channel as separate lists or tuples.